In [14]:
import tweepy
import json
import pandas as pd
import random
from dotenv import load_dotenv
import os
import time
from datetime import datetime
load_dotenv()

True

# Sentiment analysis de tweets respecto al plebiscito en Chile

Este script busca en tweeter, con la los privilegios de la cuenta develop gratuita, los tweets según las palabras clave, la localización y las fechas determinadas.

La información mas importante de cada tweet es guardado en un dataframe.

La API libre de twitter tiene varias limitaciones, entre ella la fecha:

    Keep in mind that the search index has a 7-day limit. In other words, no tweets will be found for a date older than one week.
    
No se puede traer tweets mas antiguos que una semana. Ademas del peso sobre la cantidad de tweets, que cuelga la API por 15 minutos

##### Definición de palabras clave (_keywords_) a buscar dentro del texto de los tweets:

In [2]:
keywords_list = ['Constituyente',
'Convención Constituyente',
'Convención Constitucional',
'Asamblea Constituyente',
'plebiscito',
'referéndum',
'constitución chilena',
'Acuerdo por la paz social y la Nueva Constitución',
'Vamos por Chile',
'Unión Demócrata Independiente',
'Constituyentes Apartidistas',
'Independientes No Neutrales',
'Lista del Pueblo',
'Apruebo Dignidad',
'Lista del Apruebo',
'Pueblo Constituyente',
'Chile Digno',
'Chile Unido',
'Unidos por Chile'
'Chile Libre',
'Coordinadora Constituyente Plurinacional y Popular',
'Somos región',
'Colectivo Socialista',
'Independientes por una Nueva Constitución',
'Movimientos sociales constituyentes',
'Colectivo del Apruebo',
'Wallmapur',
'Elisa Loncón',
'Campaña de desprestigio',
'rechazo del prebiscito de 2020',
'Carmen Gloria Valladares Moyano',
'Elisa Loncon',
'Destitucion de Elisa Loncon',
'#DestitucionDeElisaLoncon',
'juicio político',
'Jaime Bassa',
'Marcela Cubillos',
'Jaime Mañalich',
'Autodesprestigio',
'María Elisa Quinteros Cáceres',
'Gaspar Dominguez Donoso']

#### Funciones
El script tiene dos funciones principales:
- Activación de la API con las credenciales
- Búsqueda de los tweets con lo parámetros definidos

##### Función Activación

In [3]:
# Función de generación de la API
def oauth():
    #Pass in our twitter API authentication key
    consumer_key = os.getenv('CONSUMER_KEY')
    consumer_secret = os.getenv('CONSUMER_SECRET')
    access_token = os.getenv('ACCESS_TOKEN')
    access_token_secret = os.getenv('ACCESS_TOKEN_SECRET')
    
    auth = tweepy.OAuth1UserHandler(
        consumer_key, consumer_secret,
        access_token, access_token_secret
    )
    #Instantiate the tweepy API
    api = tweepy.API(auth, wait_on_rate_limit=True)
    return api

##### Función Búsqueda

In [4]:
# Función de búsqueda de tweets según los keywords, cantidad y localización
def search(geo, serch_query, no_of_tweets):
    try:
        #The number of tweets we want to retrieved from the search
        tweets = api.search_tweets(
                                   q=serch_query, 
                                   geocode = geo, 
                                   count=no_of_tweets
                                  )
        #Pulling Some attributes from the tweet
        attributes_container = [[tweet.user.name, 
                                 tweet.created_at, 
                                 tweet.favorite_count, 
                                 tweet.source,  
                                 tweet.text 
                                ] for tweet in tweets]
        # Creamos las keys del dict para guardar la info de la lista attributes_containter
        keys = ["User", 
                "Date Created", 
                "Number of Likes", 
                "Source of Tweet", 
                "Tweet"]
        # Generamos el dict vacío
        dict = {}
        # Variable que se incrementa cada vez que analizamos un twit nuevo, y genera un indice en el dict
        # para diferenciarlo del anterior
        a=1
        
        for value in tweets:
            dict[a] = {}
            dict[a]['User'] = value.user.name
            dict[a]['Date Created'] = value.created_at
            dict[a]['Number of likes'] = value.favorite_count
            dict[a]['Source of Tweet'] = value.source
            dict[a]['Tweet'] = value.text
            a = a+1
        return dict
    except BaseException as e:
        print('Status Failed On,',str(e))
        return

#### Instanciamos la API

In [5]:
# Instanciamos al API
api = oauth()

#### Cargamos los parámetros de búsqueda


- `sgo`: latitud y longitud de Santiago de Chile
- `radio`: radio en km alrededor del punto
- `no_of_tweets`: Cantidad de tweets para cada keyword
- `geo`: arma el feature según lo cargado anteriormente, con la ciudad y el radio.
- `keywords_list`: Se incluye en esta sección sólo para testear. Se limita a dos palabras. Si se comenta esa línea, usa toda la lista presentada arriba)

In [10]:
sgo = '-33.4489,-70.6693'
radio = '100km'
no_of_tweets =1000
geo = str(sgo) + ',' + str(radio) 
keywords_list = ['Marcela Cubillos','Jaime Mañalich'] # Comentar esta linea para usar toda la keywords_list
df = pd.DataFrame()

#### Lanzamos la búsqueda

In [11]:
for search_query in keywords_list:
    dict = search(geo, search_query, no_of_tweets)
    df1 =pd.DataFrame.from_dict(dict,orient='index')
    frames = [df, df1]
    df = pd.concat(frames)
    time.sleep(910) # Esperamos 15 minutos + 10 segundo para no reventar la API

#### Mostramos el DF 

In [12]:
df

,User,Date Created,Number of likes,Source of Tweet,Tweet
1,🌳🌲🍊🇨🇱 Viviluc. #AprobéCC,2022-08-20 17:23:32+00:00,4,Twitter for Android,"Si crees que Marcela Cubillos, hoy en España y..."
2,🅰️ndres #h🅰️br🅰️nuev🅰️constitucion,2022-08-20 16:27:49+00:00,5,Twitter for iPhone,@mcubillossigall @gabrielboric ojalá que la ma...
3,La Tercera,2022-08-20 13:50:00+00:00,69,TweetDeck,#LTSábado - Protagonistas de la Convención | M...
4,allthuser,2022-08-20 11:09:16+00:00,0,Twitter for iPad,&lt; Marcela Cubillos [ @mcubillossigall ] : “...
5,Pablo Torrejon,2022-08-20 05:30:21+00:00,0,Twitter Web App,@Mister_Wolf_0 Oye pero esta charla la dió ebr...
...,...,...,...,...,...
10,Pauta,2022-08-19 11:45:26+00:00,0,TweetDeck,Fallo de la Corte Suprema ordena suspender alz...
11,Sergio AC 🇨🇱 🇺🇦,2022-08-18 18:10:00+00:00,0,Twitter Web App,@jmanalich 👏 alguna dda penal?...derecha despi...
12,Agricultura,2022-08-18 13:44:26+00:00,26,TweetDeck,"Jaime Mañalich afirma que pandemia está ""desat..."
13,Mario luis,2022-08-14 00:15:42+00:00,0,Twitter for iPhone,"@FranfiiVale Uds fueron por Andrés chadwic, Ví..."


#### Guardamos el dF

In [17]:
filename = 'plebiscito_' + datetime.now().strftime('%m_%d_%Y_%H:%M:%S') + '.csv'
df.to_csv(filename, index = False)